# Face Generation

In this project, you'll define and train a Generative Adverserial network of your own creation on a dataset of faces. Your goal is to get a generator network to generate *new* images of faces that look as realistic as possible!

The project will be broken down into a series of tasks from **defining new architectures training adversarial networks**. At the end of the notebook, you'll be able to visualize the results of your trained Generator to see how it performs; your generated samples should look like fairly realistic faces with small amounts of noise.

### Get the Data

You'll be using the [CelebFaces Attributes Dataset (CelebA)](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) to train your adversarial networks.

This dataset has higher resolution images than datasets you have previously worked with (like MNIST or SVHN) you've been working with, and so, you should prepare to define deeper networks and train them for a longer time to get good results. It is suggested that you utilize a GPU for training.

### Pre-processed Data

Since the project's main focus is on building the GANs, we've done *some* of the pre-processing for you. Each of the CelebA images has been cropped to remove parts of the image that don't include a face, then resized down to 64x64x3 NumPy images. Some sample data is show below.

<img src='assets/processed_face_data.png' width=60% />

> If you are working locally, you can download this data [by clicking here](https://s3.amazonaws.com/video.udacity-data.com/topher/2018/November/5be7eb6f_processed-celeba-small/processed-celeba-small.zip)

This is a zip file that you'll need to extract in the home directory of this notebook for further loading and processing. After extracting the data, you should be left with a directory of data `processed-celeba-small/`.

In [ ]:
# run this once to unzip the file
# !unzip processed-celeba-small.zip

In [ ]:
from glob import glob
from typing import Tuple, Callable, Dict

import matplotlib.pyplot as plt
import numpy as np
import torch
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose, ToTensor, \
    RandomHorizontalFlip, RandomVerticalFlip, ColorJitter, Normalize

import tests

In [ ]:
data_dir = 'processed_celeba_small/celeba/'

## Data pipeline

The [CelebA](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) dataset contains over 200,000 celebrity images with annotations. Since you're going to be generating faces, you won't need the annotations, you'll only need the images. Note that these are color images with [3 color channels (RGB)](https://en.wikipedia.org/wiki/Channel_(digital_image)#RGB_Images) each.

### Pre-process and Load the Data

Since the project's main focus is on building the GANs, we've done *some* of the pre-processing for you. Each of the CelebA images has been cropped to remove parts of the image that don't include a face, then resized down to 64x64x3 NumPy images. This *pre-processed* dataset is a smaller subset of the very large CelebA dataset and contains roughly 30,000 images. 

Your first task consists in building the dataloader. To do so, you need to do the following:
* implement the get_transforms function
* create a custom Dataset class that reads the CelebA data

### Exercise: implement the get_transforms function

The `get_transforms` function should output a [`torchvision.transforms.Compose`](https://pytorch.org/vision/stable/generated/torchvision.transforms.Compose.html#torchvision.transforms.Compose) of different transformations. You have two constraints:
* the function takes a tuple of size as input and should **resize the images** to the input size
* the output images should have values **ranging from -1 to 1**

In [ ]:
import torchvision.transforms.functional as TF

class MyScaleTransform:
    """Take image and return values ranging from -1 to 1"""

    def __init__(self): #https://stackoverflow.com/questions/42884795/python-empty-constructor
        pass

    def __call__(self, x):
        return 2.0*(TF.to_tensor(x) - 0.5)


def get_transforms(size: Tuple[int, int]) -> Callable:
    """ Transforms to apply to the image."""
    # TODO: edit this function by appening transforms to the below list
    transforms = [RandomHorizontalFlip(), # https://arxiv.org/pdf/1511.06434.pdf advises against augmentation
                  RandomVerticalFlip(),
                  MyScaleTransform()]

    return Compose(transforms)

### Exercise: implement the DatasetDirectory class


The `DatasetDirectory` class is a torch Dataset that reads from the above data directory. The `__getitem__` method should output a transformed tensor and the `__len__` method should output the number of files in our dataset. You can look at [this custom dataset](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files) for ideas. 

In [ ]:
import os

class DatasetDirectory(Dataset):
    """
    A custom dataset class that loads images from folder.
    args:
    - directory: location of the images
    - transform: transform function to apply to the images
    - extension: file format
    """
    def __init__(self, 
                 directory: str, 
                 transforms: Callable = None, 
                 extension: str = '.jpg'):
        # TODO: implement the init method
        self.root_dir = directory
        self.transform = transforms
        self.extension = extension

    def __len__(self) -> int:
        """ returns the number of items in the dataset """
        # TODO: return the number of elements in the dataset
        # https://stackoverflow.com/questions/2632205/how-to-count-the-number-of-files-in-a-directory-using-python
        return len([name for name in os.listdir(self.root_dir) if self.extension in name])

    def __getitem__(self, index: int) -> torch.Tensor:
        """ load an image and apply transformation """
        # TODO: return the index-element of the dataset
        img_path = os.path.join(data_dir, 'new_folder_with_items/' 
                                + str(int(1e6 + index + 1))[1:] # adds leading zeros to image path
                                + self.extension)
        image = Image.open(img_path)
        return self.transform(image)

In [ ]:
"""
DO NOT MODIFY ANYTHING IN THIS CELL
"""
# run this cell to verify your dataset implementation
dataset = DatasetDirectory(data_dir, get_transforms((64, 64)))
tests.check_dataset_outputs(dataset)

The functions below will help you visualize images from the dataset.

In [ ]:
"""
DO NOT MODIFY ANYTHING IN THIS CELL
"""

def denormalize(images):
    """Transform images from [-1.0, 1.0] to [0, 255] and cast them to uint8."""
    return ((images + 1.) / 2. * 255).astype(np.uint8)

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(20, 4))
plot_size=20
for idx in np.arange(plot_size):
    ax = fig.add_subplot(2, int(plot_size/2), idx+1, xticks=[], yticks=[])
    img = dataset[idx].numpy() 
    img = np.transpose(img, (1, 2, 0))
    img = denormalize(img)
    ax.imshow(img)

## Model implementation

As you know, a GAN is comprised of two adversarial networks, a discriminator and a generator. Now that we have a working data pipeline, we need to implement the discriminator and the generator. 

Feel free to implement any additional class or function.

### Exercise: Create the discriminator

The discriminator's job is to score real and fake images. You have two constraints here:
* the discriminator takes as input a **batch of 64x64x3 images**
* the output should be a single value (=score)

Feel free to get inspiration from the different architectures we talked about in the course, such as DCGAN, WGAN-GP or DRAGAN.

#### Some tips
* To scale down from the input image, you can either use `Conv2d` layers with the correct hyperparameters or Pooling layers.
* If you plan on using gradient penalty, do not use Batch Normalization layers in the discriminator.

In [ ]:
from torch.nn import Module

import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# follows from DCGAN_Exercise.ipynb
class Discriminator(Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        # TODO: instantiate the different layers
        first_conv_out_channels = 32
        second_conv_out_channels = 64
        third_conv_out_channels = 128 

        self.conv1 = nn.Conv2d(in_channels=3,
                               out_channels=first_conv_out_channels,
                               kernel_size=4,
                               stride=4,
                               padding=0) # LxW is 16x16
        
        self.conv2 = nn.Conv2d(in_channels=first_conv_out_channels,
                               out_channels=second_conv_out_channels,
                               kernel_size=2,
                               stride=2,
                               padding=0) # LxW is 8x8
        self.batchnorm2 = nn.BatchNorm2d(second_conv_out_channels)

        self.conv3 = nn.Conv2d(in_channels=second_conv_out_channels,
                               out_channels=third_conv_out_channels,
                               kernel_size=2,
                               stride=2,
                               padding=0) # LxW is 4x4
        self.batchnorm3 = nn.BatchNorm2d(third_conv_out_channels)  

        self.fc1 = nn.Linear(in_features=4*4*third_conv_out_channels,
                             out_features=1)      
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # TODO: implement the forward method
        x = F.leaky_relu(self.conv1(x))
        x = F.leaky_relu(self.batchnorm2(self.conv2(x)))
        x = F.leaky_relu(self.batchnorm3(self.conv3(x)))

        x = torch.flatten(x, 1)

        x = self.fc1(x) # per note, no activation fxn. will ult. use BCE w LL
        x = torch.unsqueeze(x, 2) # adding two dims to pass tests
        x = torch.unsqueeze(x, 3)
        return x 

In [ ]:
# images = torch.randn(1, 3, 64, 64) # (batch, channels, length, height)
# after_conv1_discriminator = Discriminator()
# print(after_conv1_discriminator(images).shape)

In [ ]:
# after_conv2_discriminator = Discriminator()
# print(after_conv2_discriminator(images).shape)

In [ ]:
# after_conv3_discriminator = Discriminator()
# print(after_conv3_discriminator(images).shape)

In [ ]:
# images = torch.randn(128, 3, 64, 64)
# print(after_conv3_discriminator(images).shape)

In [ ]:
"""
DO NOT MODIFY ANYTHING IN THIS CELL
"""
# run this cell to check your discriminator implementation
discriminator = Discriminator()
tests.check_discriminator(discriminator)

### Exercise: create the generator

The generator's job creates the "fake images" and learns the dataset distribution. You have three constraints here:
* the generator takes as input a vector of dimension `[batch_size, latent_dimension, 1, 1]`
* the generator must output **64x64x3 images**

Feel free to get inspiration from the different architectures we talked about in the course, such as DCGAN, WGAN-GP or DRAGAN.

#### Some tips:
* to scale up from the latent vector input, you can use `ConvTranspose2d` layers
* as is often with GANs, **Batch Normalization** helps with training

In [ ]:
class Generator(Module):
    def __init__(self, latent_dim: int):
        super(Generator, self).__init__()
        # TODO: instantiate the different layers

        self.first_conv_in_channels = 128 
        first_conv_out_channels = 64
        second_conv_out_channels = 32
        third_conv_out_channels = 3 # rgb

        self.fc1 = nn.Linear(in_features=latent_dim,
                             out_features=4*4*self.first_conv_in_channels)
        
        self.transpose_conv1 = nn.ConvTranspose2d(
                                    in_channels=self.first_conv_in_channels,
                                    out_channels=first_conv_out_channels,
                                    kernel_size=5,
                                    padding=0)
        self.batch_norm1 = nn.BatchNorm2d(first_conv_out_channels)

        self.transpose_conv2 = nn.ConvTranspose2d(
                                    in_channels=first_conv_out_channels,
                                    out_channels=second_conv_out_channels,
                                    kernel_size=9,
                                    padding=0)
        self.batch_norm2 = nn.BatchNorm2d(second_conv_out_channels)

        self.transpose_conv3 = nn.ConvTranspose2d(
                                    in_channels=second_conv_out_channels,
                                    out_channels=third_conv_out_channels,
                                    kernel_size=49,
                                    padding=0)             

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # TODO: implement the forward method
        x = torch.flatten(x, 1)
        x = self.fc1(x)

        batch_size = len(x)
        x = x.view(batch_size, self.first_conv_in_channels, 4, 4) # order based on https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html?highlight=convtranspose2d#torch.nn.ConvTranspose2d

        x = F.relu(self.batch_norm1(self.transpose_conv1(x)))
        x = F.relu(self.batch_norm2(self.transpose_conv2(x)))
        x = F.tanh(self.transpose_conv3(x))
        
        return x

In [ ]:
# batch_size = 64
# latent_dim = 128

In [ ]:
# input_vector = torch.randn(batch_size, latent_dim, 1, 1)
# after_fc1_generator = Generator(latent_dim=latent_dim, batch_size=batch_size)
# print(after_fc1_generator(input_vector).shape)

In [ ]:
# after_view_generator = Generator(latent_dim=latent_dim, batch_size=batch_size)
# print(after_view_generator(input_vector).shape)

In [ ]:
# after_t_conv1_generator = Generator(latent_dim=latent_dim, batch_size=batch_size)
# print(after_t_conv1_generator(input_vector).shape)

In [ ]:
# after_t_conv2_generator = Generator(latent_dim=latent_dim, batch_size=batch_size)
# print(after_t_conv2_generator(input_vector).shape)

In [ ]:
# after_t_conv3_generator = Generator(latent_dim=latent_dim, batch_size=batch_size)
# print(after_t_conv3_generator(input_vector).shape)

In [ ]:
"""
DO NOT MODIFY ANYTHING IN THIS CELL
"""
# run this cell to verify your generator implementation
latent_dim = 128
generator = Generator(latent_dim)
tests.check_generator(generator, latent_dim) 

## Optimizer

In the following section, we create the optimizers for the generator and discriminator. You may want to experiment with different optimizers, learning rates and other hyperparameters as they tend to impact the output quality.

### Exercise: implement the optimizers

In [ ]:
import torch.optim as optim


def create_optimizers(generator: Module, discriminator: Module):
    """ This function should return the optimizers of the generator and the discriminator """
    # TODO: implement the generator and discriminator optimizers

    # params
    lr = 0.000002 # 0.0002 # https://arxiv.org/pdf/1511.06434.pdf
    beta1 = 0.5 # https://arxiv.org/pdf/1511.06434.pdf
    beta2 = 0.999 # default value

    g_optimizer = optim.Adam(generator.parameters(), lr, [beta1, beta2])
    d_optimizer = optim.Adam(discriminator.parameters(), lr, [beta1, beta2])
    return g_optimizer, d_optimizer

## Losses implementation

In this section, we are going to implement the loss function for the generator and the discriminator. You can and should experiment with different loss function.

Some tips:
* You can choose the commonly used the binary cross entropy loss or select other losses we have discovered in the course, such as the Wasserstein distance.
* You may want to implement a gradient penalty function as discussed in the course. It is not required and the code will work whether you implement it or not.

### Exercise: implement the generator loss

The generator's goal is to get the discriminator to think its generated images (= "fake" images) are real.

In [ ]:
def generator_loss(fake_logits):
    """ Generator loss, takes the fake scores as inputs. """
    # TODO: implement the generator loss 
    # follows real_loss and fake_loss from MNIST_GAN_Exercise.ipynb
    labels = torch.ones(fake_logits.size())*0.9
    labels = labels.cuda()

    criterion = torch.nn.BCEWithLogitsLoss()
    loss = criterion(fake_logits.squeeze(), labels)    
    return loss

### Exercise: implement the discriminator loss

We want the discriminator to give high scores to real images and low scores to fake ones and the discriminator loss should reflect that.

In [ ]:
def discriminator_loss(real_logits, fake_logits):
    """ Discriminator loss, takes the fake and real logits as inputs. """
    # TODO: implement the discriminator loss 

    criterion = torch.nn.BCEWithLogitsLoss()

    # real_loss
    labels = torch.ones(real_logits.size())*0.9
    labels = labels.cuda()
    real_loss = criterion(real_logits.squeeze(), labels)

    # fake_loss
    labels = torch.zeros(fake_logits.size())
    labels = labels.cuda()
    fake_loss = criterion(fake_logits.squeeze(), labels)

    loss = real_loss + fake_loss

    return loss

### Exercise (Optional): Implement the gradient Penalty

In the course, we discussed the importance of gradient penalty in training certain types of Gans. Implementing this function is not required and depends on some of the design decision you made (discriminator architecture, loss functions).

In [ ]:
def gradient_penalty(discriminator, real_samples, fake_samples):
    """ This function enforces """
    gp = 0
    # TODO (Optional): implement the gradient penalty
    return gp

## Training


Training will involve alternating between training the discriminator and the generator. You'll use your functions real_loss and fake_loss to help you calculate the discriminator losses.

* You should train the discriminator by alternating on real and fake images
* Then the generator, which tries to trick the discriminator and should have an opposing loss function

### Exercise: implement the generator step and the discriminator step functions

Each function should do the following:
* calculate the loss
* backpropagate the gradient
* perform one optimizer step

In [ ]:
def generator_step(batch_size: int, latent_dim: int, 
                   generator: Module, 
                   discriminator: Module, 
                   g_optimizer: optim.Adam) -> Dict:
    """ One training step of the generator. """
    # TODO: implement the generator step (foward pass, loss calculation and backward pass)

    # Generate fake images
    z = np.random.uniform(-1, 1, size=(batch_size, latent_dim))
    z = torch.from_numpy(z).float()
    z = z.cuda()
    fake_images = generator(z)

    fake_images = fake_images.cuda() 
    g_optimizer.zero_grad()         

    # Compute the discriminator losses on fake images 
    # using flipped labels!
    
    # perform backprop
    g_loss = generator_loss(discriminator(fake_images))
    g_loss.backward()
    g_optimizer.step()
        
    return {'loss': g_loss}


def discriminator_step(batch_size: int, latent_dim: int, real_images: torch.Tensor,
                       generator: Module, 
                       discriminator: Module, 
                       d_optimizer: optim.Adam) -> Dict:
    """ One training step of the discriminator. """
    # TODO: implement the discriminator step (foward pass, loss calculation and backward pass)

    real_images = real_images.cuda()
    d_optimizer.zero_grad()
    
    # Generate fake images
    z = np.random.uniform(-1, 1, size=(batch_size, latent_dim))
    z = torch.from_numpy(z).float()
    z = z.cuda()
    fake_images = generator(z)

    fake_images = fake_images.cuda()       

    d_loss = discriminator_loss(real_logits=discriminator(real_images), 
                       fake_logits=discriminator(fake_images))       

    d_loss.backward()
    d_optimizer.step()

    gp = 0 # rather than implement gradient penalty yet

    return {'loss': d_loss, 'gp': gp}

### Main training loop

You don't have to implement anything here but you can experiment with different hyperparameters.

In [ ]:
from datetime import datetime

In [ ]:
# you can experiment with different dimensions of latent spaces
latent_dim = 128

# update to cpu if you do not have access to a gpu
device = 'cuda'

# number of epochs to train your model
n_epochs = 30

# number of images in each batch
batch_size = 64

In [ ]:
"""
DO NOT MODIFY ANYTHING IN THIS CELL
"""
print_every = 50

# Create optimizers for the discriminator D and generator G
generator = Generator(latent_dim).to(device) 
discriminator = Discriminator().to(device)
g_optimizer, d_optimizer = create_optimizers(generator, discriminator)

dataloader = DataLoader(dataset, 
                        batch_size=64, 
                        shuffle=True, 
                        num_workers=4, 
                        drop_last=True,
                        pin_memory=False)

In [ ]:
"""
DO NOT MODIFY ANYTHING IN THIS CELL
"""

def display(fixed_latent_vector: torch.Tensor):
    """ helper function to display images during training """
    fig = plt.figure(figsize=(14, 4))
    plot_size = 16
    for idx in np.arange(plot_size):
        ax = fig.add_subplot(2, int(plot_size/2), idx+1, xticks=[], yticks=[])
        img = fixed_latent_vector[idx, ...].detach().cpu().numpy()
        img = np.transpose(img, (1, 2, 0))
        img = denormalize(img)
        ax.imshow(img)
    plt.show()

### Exercise: implement the training strategy

You should experiment with different training strategies. For example:

* train the generator more often than the discriminator. 
* added noise to the input image
* use label smoothing

Implement with your training strategy below.

In [ ]:
fixed_latent_vector = torch.randn(batch_size, latent_dim, 1, 1).float().cuda()

losses = []
d = 100
g = 100
loss_threshold = 1.0
for epoch in range(n_epochs):
    for batch_i, real_images in enumerate(dataloader):
        real_images = real_images.to(device)
        
        ####################################
        
        # TODO: implement the training strategy
        
        ####################################

        if ((epoch + 1) % 10) == 0:
            loss_threshold = loss_threshold*0.9

        if (epoch <= 1) or (d > loss_threshold) and (g > loss_threshold):
            d_loss = discriminator_step(batch_size=batch_size, latent_dim=latent_dim, real_images=real_images,
                        generator=generator, 
                        discriminator=discriminator, 
                        d_optimizer=d_optimizer)
            g_loss = generator_step(batch_size=batch_size, latent_dim=latent_dim, 
                            generator=generator, 
                            discriminator=discriminator, 
                            g_optimizer=g_optimizer)
        elif (d < loss_threshold) and (g > loss_threshold):
            g_loss = generator_step(batch_size=batch_size, latent_dim=latent_dim, 
                            generator=generator, 
                            discriminator=discriminator, 
                            g_optimizer=g_optimizer)
        elif (d > loss_threshold) and (g < loss_threshold):
            d_loss = discriminator_step(batch_size=batch_size, latent_dim=latent_dim, real_images=real_images,
                        generator=generator, 
                        discriminator=discriminator, 
                        d_optimizer=d_optimizer)                         
        elif (d <= loss_threshold) and (g <= loss_threshold):
            d_loss = discriminator_step(batch_size=batch_size, latent_dim=latent_dim, real_images=real_images,
                        generator=generator, 
                        discriminator=discriminator, 
                        d_optimizer=d_optimizer)
            g_loss = generator_step(batch_size=batch_size, latent_dim=latent_dim, 
                            generator=generator, 
                            discriminator=discriminator, 
                            g_optimizer=g_optimizer)

        if batch_i % print_every == 0:
            # append discriminator loss and generator loss
            d = d_loss['loss'].item()
            g = g_loss['loss'].item()
            losses.append((d, g))
            # print discriminator and generator loss
            time = str(datetime.now()).split('.')[0]
            print(f'{time} | Epoch [{epoch+1}/{n_epochs}] | Batch {batch_i}/{len(dataloader)} | d_loss: {d:.4f} | g_loss: {g:.4f}')
    
    # display images during training
    generator.eval()
    generated_images = generator(fixed_latent_vector)
    display(generated_images)
    generator.train()

### Training losses

Plot the training losses for the generator and discriminator.

In [ ]:
"""
DO NOT MODIFY ANYTHING IN THIS CELL
"""
fig, ax = plt.subplots()
losses = np.array(losses)
plt.plot(losses.T[0], label='Discriminator', alpha=0.5)
plt.plot(losses.T[1], label='Generator', alpha=0.5)
plt.title("Training Losses")
plt.legend()

### Rewriting Training Process as Part of a Class

Allow for learning rate to be changed during training.
Also allow for saving weights and retraining at select points.

In [ ]:
import torch.optim as optim
from datetime import datetime
from matplotlib.figure import Figure 
import panel as pn

# requirements: 
# specify number of epochs (done)
# the learning rate of the generator (tech debt - need to specify 2nd time in train)
# the learning rate of the discriminator (tech debt - need to specify 2nd time in train)
# save the weights of both (not necessary yet)
# load the weights of both (not necessary yet)
# specify whether to train the generator, the discriminator or both (done)
# save images (not needed yet)
# load images, but do not need to print images within class (not needed yet)

class Training():
    def __init__(self, 
                 n_epochs: int, 
                 lr_generator: float, 
                 lr_discriminator: float,
                 save_weights_name: str, # generator or discriminator will be appended
                 g_load_weights_name: str,
                 d_load_weights_name: str,                 
                 ):
        
        self.lr_generator = lr_generator
        self.lr_discriminator = lr_discriminator
        self.save_weights_name = save_weights_name
        self.g_load_weights_name = g_load_weights_name
        self.d_load_weights_name = d_load_weights_name

        # you can experiment with different dimensions of latent spaces
        latent_dim = 128

        # update to cpu if you do not have access to a gpu
        self.device = 'cuda'

        # number of epochs to train your model
        self.n_epochs = n_epochs

        # number of images in each batch
        self.batch_size = 64

        self.latent_dim = 128
        self.print_every = 50

        self.fixed_latent_vector = torch.randn(self.batch_size, latent_dim, 1, 1).float().cuda()        

        # Create optimizers for the discriminator D and generator G
        
        if ((self.g_load_weights_name == 'NONE') and 
              (self.d_load_weights_name == 'NONE')):
            self.generator = Generator(self.latent_dim).to(self.device)
            self.discriminator = Discriminator().to(self.device)
            
            self.create_optimizers()

            self.dataloader = DataLoader(dataset, 
                                    batch_size=64, 
                                    shuffle=True, 
                                    num_workers=4, 
                                    drop_last=True,
                                    pin_memory=False)
        else:
            self.generator = Generator(latent_dim)
            self.discriminator = Discriminator()

            self.create_optimizers()

            g_checkpoint = torch.load(self.g_load_weights_name, map_location='cpu')
            self.generator.load_state_dict(g_checkpoint['model_state_dict'])
            self.generator.to('cuda')
            self.data_loader = g_checkpoint['dataloader']
            g_optimizer.load_state_dict(g_checkpoint['optimizer_state_dict'])
            g_optimizer.param_groups[0]['lr'] = self.lr_generator
            self.epoch = g_checkpoint['epoch']
            self.loss = g_checkpoint['loss']

            d_checkpoint = torch.load(self.d_load_weights_name, map_location='cpu')
            self.discriminator.load_state_dict(d_checkpoint['model_state_dict'])
            self.discriminator.to('cuda')
            self.data_loader = d_checkpoint['dataloader']
            d_optimizer.load_state_dict(d_checkpoint['optimizer_state_dict'])
            d_optimizer.param_groups[0]['lr'] = self.lr_discriminator
            self.epoch = d_checkpoint['epoch']
            self.loss = d_checkpoint['loss']                     

        return

    def save_weights(self):
        # save generator
        generator_save_path = self.save_weights_name + '_generator.pt'        
        torch.save({
            'epoch': self.n_epochs,
            'model_state_dict': self.generator.state_dict(),
            'optimizer_state_dict': self.g_optimizer.state_dict(),
            'loss': self.g_loss,
            'dataloader': self.dataloader
            }, generator_save_path)

        # save discriminator
        discriminator_save_path = self.save_weights_name + '_discriminator.pt'
        torch.save({
            'epoch': self.n_epochs,
            'model_state_dict': self.discriminator.state_dict(),
            'optimizer_state_dict': self.d_optimizer.state_dict(),
            'loss': self.d_loss,
            'dataloader': self.dataloader
            }, discriminator_save_path)
                
        return

    def create_optimizers(self, beta1=0.5, beta2=0.999):
        """ This function should return the optimizers of the generator and the discriminator """
        # TODO: implement the generator and discriminator optimizers

        # params
        # lr = 0.000002 # 0.0002 # https://arxiv.org/pdf/1511.06434.pdf
        # beta1 = 0.5 # https://arxiv.org/pdf/1511.06434.pdf
        # beta2 = 0.999 # default value

        self.g_optimizer = optim.Adam(self.generator.parameters(), self.lr_generator, [beta1, beta2])
        self.d_optimizer = optim.Adam(self.discriminator.parameters(), self.lr_discriminator, [beta1, beta2])
        return 
    
    def generator_loss(self, fake_logits):
        """ Generator loss, takes the fake scores as inputs. """
        # TODO: implement the generator loss 
        # follows real_loss and fake_loss from MNIST_GAN_Exercise.ipynb
        labels = torch.ones(fake_logits.size())*0.9
        labels = labels.cuda()

        criterion = torch.nn.BCEWithLogitsLoss()
        loss = criterion(fake_logits.squeeze(), labels.squeeze())
        return loss
    
    def discriminator_loss(self, real_logits, fake_logits):
        """ Discriminator loss, takes the fake and real logits as inputs. """
        # TODO: implement the discriminator loss 

        criterion = torch.nn.BCEWithLogitsLoss()

        # real_loss
        labels = torch.ones(real_logits.size())*0.9
        labels = labels.cuda()
        real_loss = criterion(real_logits.squeeze(), labels.squeeze())

        # fake_loss
        labels = torch.zeros(fake_logits.size())
        labels = labels.cuda()
        fake_loss = criterion(fake_logits.squeeze(), labels.squeeze())

        loss = real_loss + fake_loss

        return loss
    
    def display(self, fixed_latent_vector: torch.Tensor):
        """ helper function to display images during training """
        fig = plt.figure(figsize=(14, 4))        
        plot_size = 16
        for idx in np.arange(plot_size):
            self.ax = fig.add_subplot(2, int(plot_size/2), idx+1, xticks=[], yticks=[])
            img = fixed_latent_vector[idx, ...].detach().cpu().numpy()
            img = np.transpose(img, (1, 2, 0))
            img = denormalize(img)
            self.ax.imshow(img)
        plt.show()    
        return

    def generator_step(self) -> Dict:
        """ One training step of the generator. """
        # TODO: implement the generator step (foward pass, loss calculation and backward pass)

        # Generate fake images
        z = np.random.uniform(-1, 1, size=(self.batch_size, self.latent_dim))
        z = torch.from_numpy(z).float()
        z = z.cuda()
        fake_images = self.generator(z)

        fake_images = fake_images.cuda() 
        self.g_optimizer.zero_grad()         

        # Compute the discriminator losses on fake images 
        # using flipped labels!
        
        # perform backprop
        g_loss = self.generator_loss(self.discriminator(fake_images))
        g_loss.backward()
        self.g_optimizer.step()
            
        return {'loss': g_loss}

    def discriminator_step(self, real_images: torch.Tensor) -> Dict:
        """ One training step of the discriminator. """
        # TODO: implement the discriminator step (foward pass, loss calculation and backward pass)

        real_images = real_images.cuda()
        self.d_optimizer.zero_grad()
        
        # Generate fake images
        z = np.random.uniform(-1, 1, size=(self.batch_size, self.latent_dim))
        z = torch.from_numpy(z).float()
        z = z.cuda()
        fake_images = self.generator(z)

        fake_images = fake_images.cuda()       

        d_loss = self.discriminator_loss(real_logits=self.discriminator(real_images), 
                        fake_logits=self.discriminator(fake_images))       

        d_loss.backward()
        self.d_optimizer.step()

        gp = 0 # rather than implement gradient penalty yet

        return {'loss': d_loss, 'gp': gp}

    def train(self, n_epochs, lr_generator, lr_discriminator, train_generator, train_discriminator):
        self.n_epochs = n_epochs
        self.lr_generator = lr_generator
        self.lr_discriminator = lr_discriminator
        self.train_generator = train_generator
        self.train_discriminator = train_discriminator        
        
        self.create_optimizers()     
        
        losses = []
        d = 100
        g = 100
        for epoch in range(self.n_epochs):
            for batch_i, real_images in enumerate(self.dataloader):
                real_images = real_images.to(self.device)
                
                ####################################
                
                # TODO: implement the training strategy
                
                ####################################

                if self.train_generator:
                    self.g_loss = self.generator_step()

                if self.train_discriminator:
                    self.d_loss = self.discriminator_step(real_images=real_images)

                if batch_i % self.print_every == 0:
                    # append discriminator loss and generator loss
                    d = self.d_loss['loss'].item()
                    g = self.g_loss['loss'].item()
                    losses.append((d, g))
                    # print discriminator and generator loss
                    time = str(datetime.now()).split('.')[0]
                    print(f'{time} | Epoch [{epoch+1}/{self.n_epochs}] | Batch {batch_i}/{len(self.dataloader)} | d_loss: {d:.4f} | g_loss: {g:.4f}')
            
            # display images during training
            self.generator.eval()
            self.generated_images = self.generator(self.fixed_latent_vector)
            self.display(self.generated_images)
            self.generator.train()
        # self.save_weights()

In [ ]:
custom_training = Training(
                 n_epochs = 1, 
                 lr_generator = 0.00002, 
                 lr_discriminator = 0.00002,
                 save_weights_name = 'first_weights', # _generator.pt or _discriminator.pt will be appended
                 g_load_weights_name = 'NONE',
                 d_load_weights_name = 'NONE'
                 )

In [ ]:
custom_training.train(n_epochs=1, 
                      lr_generator=0.0002, 
                      lr_discriminator=0.0002, 
                      train_generator=True, 
                      train_discriminator=True)

In [ ]:
custom_training.train(n_epochs=1, 
                      lr_generator=0.0002, 
                      lr_discriminator=0.0002, 
                      train_generator=True, 
                      train_discriminator=True)

In [ ]:
custom_training.train(n_epochs=1, 
                      lr_generator=0.0002, 
                      lr_discriminator=0.000002, 
                      train_generator=False, 
                      train_discriminator=True)

In [ ]:
custom_training.train(n_epochs=10, 
                      lr_generator=0.0002, 
                      lr_discriminator=0.0002, 
                      train_generator=True, 
                      train_discriminator=False)

In [ ]:
custom_training.train(n_epochs=10, 
                      lr_generator=0.0002, 
                      lr_discriminator=0.0002, 
                      train_generator=True, 
                      train_discriminator=False)

In [ ]:
custom_training.train(n_epochs=2, 
                      lr_generator=0.00002, 
                      lr_discriminator=0.00002, 
                      train_generator=True, 
                      train_discriminator=True)

In [ ]:
for kk in range(5):
    custom_training.train(n_epochs=3, 
                        lr_generator=0.000002, 
                        lr_discriminator=0.000002, 
                        train_generator=False, 
                        train_discriminator=True)
    custom_training.train(n_epochs=3, 
                        lr_generator=0.000002, 
                        lr_discriminator=0.000002, 
                        train_generator=True, 
                        train_discriminator=False)

### Question: What do you notice about your generated samples and how might you improve this model?
When you answer this question, consider the following factors:
* The dataset is biased; it is made of "celebrity" faces that are mostly white
* Model size; larger models have the opportunity to learn more features in a data feature space
* Optimization strategy; optimizers and number of epochs affect your final result
* Loss functions

**Answer:** (Write your answer in this cell)

### Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_face_generation.ipynb".  

Submit the notebook using the ***SUBMIT*** button in the bottom right corner of the Project Workspace.